# **Trabalho Prático 1 - Geometria Computacional**


### **Universidade Federal de Minas Gerais - Instituto de Ciências Exatas**

#### Departamento de Ciência da Computação

DCC207 - Algoritmos 2

Prof. Renato Vimieiro


  
**Alunos:** Daniel Barbosa, Frederico Baker e Victor Prates

O presente trabalho visa a aplicação de algoritmos de geometria computacional para solucionar um problema de classificação em aprendizado de máquina supervisionado. Em especial, o problema de classificação a ser abordado se refere à triagem de pacientes em um serviço de saúde. A meta é criar um modelo de classificação usando técnicas como envoltória convexa e detecção de interseções entre segmentos.

No diretório ./models foram implementadas estruturas para representar pontos e segmentos, além das seguintes operações:
* Determinar a orientação relativa de dois segmentos
* Determinar se um ponto está sobre um segmento
* Determinar se dois segmentos se interceptam

In [4]:
from models.models import Point
from models.models import Segment

# 1. Envoltória convexa

# 2. Varredura linear

Aqui, vamos implementar um algoritmo que verifica se algum segment de uma envoltória A intercepta algum segmento de uma envoltória B.

Se há pelo o menos uma interceptação (ou seja, os dados não são linearmente separáveis), o algoritmo deve retornar que não é possível criar um modelo linear para tal conjunto de dados.

Se não há interceptação (ou seja, os dados são linearmente separáveis), o algoritmo deve retornar o modelo linear para tal conjunto de dados.

Primeiro, vamos tratar de um caso específico:

- Todos os segmentos de uma envoltória convexa se intersectam. Por isso, o algoritmo que impementaremos desconsiderará interseções que ocorrem exatamente na extremidade de um segmento com a extremidade de outro segmento.
    
    - Isso não atrapalhará na separabilidade das envoltórias, pois, se uma extremidade de envoltória intercepta uma extremidade da outra envoltória, essa extremidade em comum das envoltórias será exatamente o ponto pelo qual a separação linear passará
    
    - Assim, garantiremos que não haverão interseções entre segmentos de uma mesma envoltória

Embaixo vemos a implementação desse algorimto que verifica se dois segmentos intersectam em quaisquer pontos que não sejam suas extremidades:

In [5]:
def intersectSegments(seg1, seg2):
    # If the segments share an endpoint, they don't "intersect" 
    if (seg1.origin == seg2.origin) or (seg1.origin == seg2.destination) or \
       (seg1.destination == seg2.origin) or (seg1.destination == seg2.destination):
        return False
    
    return seg1.segmentIntersect(seg2)

Agora, vamos escrever alguns testes para esse código:

In [6]:
def test_intersectSegments():
    # Test Case 1: Segments that don't intersect
    seg1 = Segment(Point(0,0), Point(1,1))
    seg2 = Segment(Point(2,2), Point(3,3))
    assert not intersectSegments(seg1, seg2), "Test Case 1 Failed"
    
    # Test Case 2: Segments that intersect
    seg1 = Segment(Point(0,0), Point(2,2))
    seg2 = Segment(Point(2,0), Point(0,2))
    assert intersectSegments(seg1, seg2), "Test Case 2 Failed"
    
    # Test Case 3: Segments that share an endpoint but don't "intersect"
    seg1 = Segment(Point(0,0), Point(2,2))
    seg2 = Segment(Point(2,2), Point(4,4))
    assert not intersectSegments(seg1, seg2), "Test Case 3 Failed"
    
    # Test Case 4: Segments that are colinear but don't overlap
    seg1 = Segment(Point(0,0), Point(2,2))
    seg2 = Segment(Point(3,3), Point(4,4))
    assert not intersectSegments(seg1, seg2), "Test Case 4 Failed"
    
    # Test Case 5: Segments that are colinear and overlap
    seg1 = Segment(Point(0,0), Point(3,3))
    seg2 = Segment(Point(2,2), Point(4,4))
    assert intersectSegments(seg1, seg2), "Test Case 5 Failed"

    # Test Case 6: Vertical segments that don't intersect
    seg1 = Segment(Point(0, 0), Point(0, 2))
    seg2 = Segment(Point(0, 3), Point(0, 5))
    assert not intersectSegments(seg1, seg2), "Test Case 6 Failed"
    
    # Test Case 7: Vertical segments that do intersect
    seg1 = Segment(Point(0, 0), Point(0, 3))
    seg2 = Segment(Point(0, 2), Point(0, 4))
    assert intersectSegments(seg1, seg2), "Test Case 7 Failed"

    print("All test cases passed!")
    
test_intersectSegments()


All test cases passed!


Notes to self:

- lembrar de verificar os casos de exceção do algoritmo de varredura linear:
    - Três segmentos não se interceptam em um único ponto
    - Não há segmentos puramente verticais

# 3. Verificação de separabilidade

# 4. Construção do modelo e classificador

# 5. Método para computação das métricas

# 6. Conclusão dos experimentos

# 7. Conclusão do relatório